<a href="https://colab.research.google.com/github/shivendrra/enigma-1.5b/blob/main/enigma/TrainEnigma.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Train file for enigma model

- Contains K-mer tokenizer, k=4, can be changed though
- Train data is available on huggingface repo: [hf/engima-1.5b](https://huggingface.co/shivendrra/enigma-1.5b)
- For now, trainig decoder-based model only
- More about this on github repo: [github/enigma-1.5b](https://github.com/shivendrra/enigma-1.5b)
- Saves model after training in '.pth' & '.safetensors' file for later use
- Generate function works fine

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
class PerCharTokenizer:
  """
  Args:
    - chars (list): all bases along with special tokens represented as characters
    - vocab_size (int): size of vocabulary

  Working:
    - vocab contains all the bases and ['P', 'M', 'U'] as padding, mask and unknown token
    - encode(): iterates over each character a time and the looks up for the position in vocab
      and returns it's position as integer
    - decode(): takes input of a list of integers and returns the specific item from vocab
  """
  def __init__(self):
    super().__init__()
    self.chars = ['\n', 'A', 'T', 'G', 'C', 'P', 'M', 'U', ' ']
    self.vocab_size = len(self.chars)
    self.string_to_index = {ch: i for i, ch in enumerate(self.chars)}
    self.index_to_string = {i: ch for i, ch in enumerate(self.chars)}

  def encode(self, string):
    encoded = []
    for char in string:
      if char in self.string_to_index:
        encoded.append(self.string_to_index[char])
      else:
        special_index = len(self.string_to_index)
        self.string_to_index[char] = special_index
        self.index_to_string[special_index] = char
        encoded.append(special_index)
    return encoded

  def decode(self, integer):
    decoded = []
    for i in integer:
      if i in self.index_to_string:
        decoded.append(self.index_to_string[i])
      else:
        continue
    return ''.join(decoded)

In [ ]:
from tqdm import tqdm
import json

class KMerTokenizer:
  def __init__(self, k_mers: int=4):
    self.k_mers = k_mers
    self.vocab = {}
    self.id_to_token = []
    self.token_to_id = {}

  def tokenize_sequence(self, sequence):
    kmers = [sequence[i:i+self.k_mers] for i in tqdm(range(0, len(sequence), self.k_mers), desc="tokenizing k-mers")]
    return kmers

  def build_vocab(self, sequences):
    all_kmers = []
    for sequence in sequences:
      all_kmers.extend(self.tokenize_sequence(sequence))
    token_count = {}
    for kmer in all_kmers:
      if kmer in token_count:
        token_count[kmer] += 1
      else:
        token_count[kmer] = 1
    sorted_tokens = sorted(token_count.items(), key=lambda x: x[1], reverse=True)
    for token, _ in sorted_tokens:
      self.token_to_id[token] = len(self.token_to_id)
      self.id_to_token.append(token)
    self.vocab = self.token_to_id

  def encode(self, sequence):
    encoded_sequence = []
    kmers = self.tokenize_sequence(sequence)
    for kmer in tqdm(kmers, desc="encoding sequences"):
      if kmer in self.token_to_id:
        encoded_sequence.append(self.token_to_id[kmer])
      else:
        encoded_sequence.append(len(self.vocab))
    return encoded_sequence

  def decode(self, encoded_sequence):
    decoded_sequence = [self.id_to_token[token_id] for token_id in encoded_sequence]
    return decoded_sequence

  def save_model(self, model_path):
    vocab_file = f"{model_path}/base_{self.k_mers}k.json"
    with open(vocab_file, 'w') as f:
      json.dump(self.vocab, f)

  def load_model(self, path):
    assert path.endswith('.json')
    with open(path, 'r') as f:
      vocab = json.load(f)

    self.vocab = vocab
    self.token_to_id = self.vocab
    self.vocab_size = len(vocab)

In [ ]:
token = KMerTokenizer(k_mers=5)
token.load_model('/content/drive/MyDrive/base_5k.json')
print(f"vocab size: {len(token.vocab)}")

vocab size: 2282


In [3]:
token = PerCharTokenizer()

In [4]:
import torch

# importing the data
file_path = '/content/drive/MyDrive/training data/consolidated_00.txt'
with open(file_path, 'r', encoding='utf-8') as file:
  input_seq = file.read()
file.close()

print(f"{(len(input_seq)/1e9):.2f} billion letters")

1.94 billion letters


In [5]:
# Train and test splits
n = int(0.9*len(input_seq)) # first 90% will be train, rest val
train_data = torch.tensor(token.encode(input_seq[:n]), dtype=torch.long)
val_data = torch.tensor(token.encode(input_seq[n:]), dtype=torch.long)

del input_seq, n
print(f"total tokens: {(len(train_data)/1e6 + len(val_data)/1e6):.0f} million")
print(f"train data {(len(train_data)/1e6):.0f}million, val data {(len(val_data)/1e6):.0f}million")

total tokens: 1942 million
train data 1748million, val data 194million


In [6]:
# hyperparams
batch_size = 10
block_size = 256
max_iters = 5000
eval_interval = 100
learning_rate = 3e-6
eval_iters = 250
d_model = 512
n_layers = 12
n_head = 18
dropout = 0.2
norm_eps = 1e-5

In [7]:
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'

class RMSNorm(nn.Module):
  def __init__(self, dim: int, eps: float = 1e-6):
    super().__init__()
    self.eps = eps
    self.weight = nn.Parameter(torch.ones(dim))

  def _norm(self, x):
    return x * torch.rsqrt(x.pow(2).mean(-1, keepdim=True) + self.eps)

  def forward(self, x):
    output = self._norm(x.float()).type_as(x)
    return output * self.weight

class SingleHead(nn.Module):
  def __init__(self,
      head_size: int,
      d_model: int,
      block_size: int,
      dropout: float):
    super().__init__()
    self.key = nn.Linear(d_model, head_size, bias=True)
    self.query = nn.Linear(d_model, head_size, bias=True)
    self.value = nn.Linear(d_model, head_size, bias=True)
    self.dropout = nn.Dropout(dropout)
    self.rel_pos_embd = nn.Parameter(torch.randn(block_size, block_size, head_size))
    self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

  def forward(self, x: torch.Tensor, mask: bool= False):
    B, T, C = x.shape
    key = self.key(x)
    query = self.query(x)
    scores = torch.matmul(query ,key.transpose(-2, -1)) / (key.shape[-1]**-0.5)
    rel_pos_scores = torch.einsum('btc,tvc->btv', query, self.rel_pos_embd[:T, :T])
    scores = scores + rel_pos_scores

    if mask is True:
      scores = scores.masked_fill(self.tril[:T, :T] == 0, float('-inf'))

    att_mat = F.softmax(scores, dim=-1)
    att_mat = self.dropout(att_mat)
    value = self.value(x)
    output = torch.matmul(att_mat, value)

    return output

class MultiHeadAttention(nn.Module):
  def __init__(self,
      d_model: int,
      block_size: int,
      n_head : int,
      dropout: float):
    head_size = d_model // n_head
    super().__init__()
    self.heads = nn.ModuleList([SingleHead(d_model=d_model, dropout=dropout, block_size=block_size, head_size=head_size) for _ in range(n_head)])
    self.projection = nn.Linear(head_size * n_head, d_model)
    self.dropout = nn.Dropout(dropout)

  def forward(self, x: torch.Tensor, mask: bool):
    out = torch.cat([h(x, mask) for h in self.heads], dim=-1)
    out = self.dropout(self.projection(out))

    del x
    return out

class FeedForward(nn.Module):
  def __init__(self, d_model, dropout):
    super().__init__()
    self.net = nn.Sequential(
      nn.Linear(d_model, 5 * d_model),
      nn.GELU(),
      nn.Linear(5 * d_model, d_model),
      nn.Dropout(dropout),
      )

  def forward(self, x: torch.Tensor):
    return self.net(x)

class DecoderBlock(nn.Module):
  def __init__(self, d_model: int,
        block_size: int,
        n_head: int,
        norm_eps: float,
        dropout: float):
    super().__init__()
    self.self_att = MultiHeadAttention(n_head=n_head, d_model=d_model, dropout=dropout, block_size=block_size)
    self.ffwd = FeedForward(d_model, dropout)
    self.dropout = nn.Dropout(dropout)
    self.norm = RMSNorm(d_model, eps=norm_eps)

  def forward(self, x: torch.Tensor):
    x_out = self.self_att(self.norm(x), mask=True)
    x_out = x + self.dropout(x_out)
    del x

    x = self.self_att(self.norm(x_out), mask=False)
    x = x_out + self.dropout(x)
    del x_out

    x_out = self.ffwd(self.norm(x))
    x_out = x + self.dropout(x_out)
    del x

    return x_out

class Transformer(nn.Module):
  def __init__(self, vocab_size: int):
    super().__init__()
    self.block_size = block_size
    self.token_embeddings = nn.Embedding(vocab_size, d_model)
    self.decoder = nn.Sequential(*[DecoderBlock(n_head=n_head, d_model=d_model, dropout=dropout, norm_eps=norm_eps, block_size=block_size) for _ in range(n_layers)])
    self.norm_final = RMSNorm(d_model, eps=norm_eps)
    self.linear_final = nn.Linear(d_model, vocab_size)
    self.dropout = nn.Dropout(dropout)
    self.apply(self._init_weights)

  def _init_weights(self, module):
    if isinstance(module, nn.Linear):
      torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
      if module.bias is not None:
        torch.nn.init.zeros_(module.bias.data)
    elif isinstance(module, nn.Embedding):
      torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

  def forward(self, idx, targets=None):
    B, T = idx.shape
    x = self.token_embeddings(idx)
    x = self.decoder(x)
    logits = self.linear_final(self.norm_final(x))

    if targets is None:
      loss = None

    else:
      B, T, C = logits.shape
      logits = logits.view(B*T, C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits, targets)

    return logits, loss

  @torch.no_grad()
  def generate(self, idx, max_new_tokens, temperature=1.0, top_k=None):
    self.eval()
    for _ in range(max_new_tokens):

      idx_cond = idx if idx.size(1) <= self.block_size else idx[:, -self.block_size:]
      logits, _ = self(idx_cond)
      logits = logits[:, -1, :] / temperature

      if top_k is not None:
        v, _ = torch.topk(logits, min(top_k, logits.size(-1)))
        logits[logits < v[:, [-1]]] = -float('Inf')

      probs = F.softmax(logits, dim=-1)
      idx_next = torch.multinomial(probs, num_samples=1)
      idx = torch.cat((idx, idx_next), dim=1)

    return idx

In [8]:
import timeit
start_time = timeit.default_timer()

torch.manual_seed(1200)
def get_batch(split):
  data = train_data if split == 'train' else val_data
  ix = torch.randint(len(data) - block_size, (batch_size,))
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  x, y = x.to(device), y.to(device)
  return x, y

@torch.no_grad()
def estimate_loss():
  out = {}
  model.eval()
  for split in ['train', 'val']:
    losses = torch.zeros(eval_iters)
    for k in range(eval_iters):
      X, Y = get_batch(split)
      logits, loss = model(X, Y)
      losses[k] = loss.item()
    out[split] = losses.mean()
  model.train()
  return out

vocab_size = token.vocab_size
model = Transformer(vocab_size)
# checkpoint_path = '/content/drive/MyDrive/enigma-2.5b.pth'
# checkpoint = torch.load(checkpoint_path)
# model.load_state_dict(checkpoint)
m = model.to(device)

# no of parameters
n_param = sum(p.numel() for p in m.parameters())/1e6
print(f"{n_param:.1f} million parameters")

# optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
steps = []
train_losses = []
val_losses = []

for iter in range(max_iters):

  if iter % eval_interval == 0 or iter == max_iters - 1:
    losses = estimate_loss()
    print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    steps.append(iter)
    train_losses.append(losses['train'])
    val_losses.append(losses['val'])

  xb, yb = get_batch('train')
  logits, loss = model(xb, yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

440.3 million parameters
step 0: train loss 2.1436, val loss 2.1208
step 100: train loss 1.3356, val loss 1.3435
step 200: train loss 1.3446, val loss 1.3494
step 300: train loss 1.3330, val loss 1.3357
step 400: train loss 1.3154, val loss 1.3247
step 500: train loss 1.3320, val loss 1.3381
step 600: train loss 1.3149, val loss 1.3248
step 700: train loss 1.3250, val loss 1.3344
step 800: train loss 1.3265, val loss 1.3322
step 900: train loss 1.3272, val loss 1.3337
step 1000: train loss 1.3139, val loss 1.3229
step 1100: train loss 1.3298, val loss 1.3368
step 1200: train loss 1.3210, val loss 1.3261
step 1300: train loss 1.3174, val loss 1.3218
step 1400: train loss 1.3129, val loss 1.3210


KeyboardInterrupt: 

In [9]:
end_time = timeit.default_timer()
print(f"total parameters: {n_param:.1f} billion")
print(f"trained in {((end_time - start_time)/3600):.2f}hrs")

total parameters: 440.3 billion
trained in 1.06hrs


In [10]:
input_from_user = "TGCCCTGGCTGCTCCGCATTGCAGGAGCTGCGCCCTTCCTTTC"

token_input = token.encode(input_from_user)

target = "TGCCCTGGCTGCTCCGCATTGCAGGAGCTGCGCCCTTCCTTTCCTGGCACAGGGTCCACAGCCCCGAAACCCCGTTGTGTGGGAGCTGGGCA"
target = "TGCCCTGGCTGCTCCGCATTGCAGGAGCTGCGCCCTTCCTTTCC"
# generate from the model
context = torch.tensor([token_input], dtype=torch.long, device=device)
generated_output = token.decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_output)

TGCCCTGGCTGCTCCGCATTGCAGGAGCTGCGCCCTTCCTTTCC


In [11]:
model_save_name = f'consolidated_00.pth'
path = f"/content/drive/MyDrive/{model_save_name}"
torch.save(model.state_dict(), path)

# saving safe-tensors
from safetensors.torch import save_file

model_save_name = f'consolidated_00.safetensors'
path = f"/content/drive/MyDrive/{model_save_name}"
save_file(model.state_dict(), path)

In [ ]:
!nvidia-smi